# Using transformers for CombinedCesnetDataset

### Import

In [1]:
import numpy as np
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TransformerType, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import CombinedConfig # Combined dataset MUST use CombinedConfig

from cesnet_tszoo.utils.transformer import Transformer # For creating custom Transformer

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
combined_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.IP_ADDRESSES_SAMPLE, aggregation=AgreggationType.AGG_10_MINUTES, dataset_type=DatasetType.COMBINED, display_details=True)

[2025-08-17 15:46:42,141][wrapper_dataset][INFO] - Dataset is combined. Use cesnet_tszoo.configs.CombinedConfig



Dataset details:

    AgreggationType.AGG_10_MINUTES
        Time indices: range(0, 40297)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 3, 49, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 50, 52, tzinfo=datetime.timezone.utc))

    SourceType.IP_ADDRESSES_SAMPLE
        Time series indices: [ 11  20 101 103 118 ... 2003134 2008461 2011839 2022235 2044888], Length=1000; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'n_dest_ip': 0, 'n_dest_asn': 0, 'n_dest_ports': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Transformers

- Transformers are implemented as class.
    - You can create your own or use built-in one.
- Transformer is applied after `default_values` and fillers took care of missing values.
- One transformer is used for all time series.
- Transformer must implement `transform`.
- Transformer must implement `partial_fit` (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- To use transformer, train set must be implemented (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- You can change used transformer later with `update_dataset_config_and_initialize` or `apply_transformer`.

#### Built-in

In [4]:
# Options

## Supported
TransformerType.STANDARD_SCALER
TransformerType.L2_NORMALIZER
TransformerType.LOG_TRANSFORMER
TransformerType.MAX_ABS_SCALER
TransformerType.MIN_MAX_SCALER

<TransformerType.MIN_MAX_SCALER: 'min_max_scaler'>

In [5]:
config = CombinedConfig(train_ts=500, val_ts=None, test_ts=None, train_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=TransformerType.MIN_MAX_SCALER, nan_threshold=0.5, random_state=1500)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:42,151][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:42,188][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:42,191][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1412.48it/s]
[2025-08-17 15:46:42,557][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: min_max_scaler
        Are transformers premade: False
        Are premade transformers partial_fitted:

In [6]:
combined_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [7]:
combined_dataset.get_transformers()

Or later with:

In [8]:
combined_dataset.update_dataset_config_and_initialize(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)
# Or
combined_dataset.apply_transformer(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)

[2025-08-17 15:46:42,761][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-17 15:46:42,797][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:42,800][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 414.65it/s]
[2025-08-17 15:46:42,947][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-17 15:46:42,947][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:42,948][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-17 15:46:42,982][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:42,985][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 440.08it/s]
[2025-08-17 15:46:43,124][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-17 15:46:43,124][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:

#### Custom

You can create your own custom transformer. It is recommended to derive from Transformer base class.

In [9]:
class CustomTransformer(Transformer):
    def __init__(self):
        super().__init__()
        
        self.max = None
        self.min = None
    
    def transform(self, data):
        return (data - self.min) / (self.max - self.min)
    
    def fit(self, data):
        self.partial_fit(data)
    
    def partial_fit(self, data):
        
        if self.max is None and self.min is None:
            self.max = np.max(data, axis=0)
            self.min = np.min(data, axis=0)
            return
        
        temp_max = np.max(data, axis=0)
        temp = np.vstack((self.max, temp_max)) 
        self.max = np.max(temp, axis=0)
        
        temp_min = np.min(data, axis=0)
        temp = np.vstack((self.min, temp_min)) 
        self.min = np.min(temp, axis=0)            

In [10]:
config = CombinedConfig(train_ts=500, val_ts=None, test_ts=None, train_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:43,135][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:43,169][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:43,172][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1563.88it/s]
[2025-08-17 15:46:43,494][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: False
        Are premade transformers par

In [11]:
combined_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [12]:
combined_dataset.get_transformers()

Or later with:

In [13]:
combined_dataset.update_dataset_config_and_initialize(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)
# Or
combined_dataset.apply_transformer(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)

[2025-08-17 15:46:43,676][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-17 15:46:43,711][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:43,714][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 491.74it/s]
[2025-08-17 15:46:43,838][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-17 15:46:43,838][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:43,839][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-17 15:46:43,872][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:43,875][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 485.51it/s]
[2025-08-17 15:46:44,001][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-17 15:46:44,001][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:

#### Using already fitted transformer

- When `partial_fit_initialized_transformer` is False (default value), transformer has no requirement for `partial_fit` nor for train set.

In [14]:
config = CombinedConfig(train_ts=500, val_ts=None, test_ts=None, train_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=False, workers=0)

fitted_transformer = combined_dataset.get_transformers()

[2025-08-17 15:46:44,006][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:44,092][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:44,095][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1703.78it/s]
[2025-08-17 15:46:44,389][cesnet_dataset][INFO] - Config initialized successfully.


In [15]:
config = CombinedConfig(train_ts=500, val_ts=500, test_ts=None, train_time_period=0.5, val_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:44,393][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:44,461][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:44,463][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1854.00it/s]
[2025-08-17 15:46:44,738][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 500/500 [00:00<00:00, 1544.12it/s]
[2025-08-17 15:46:45,063][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDs: [  1672 210412 450900 777928  44252 ... 1825722   11254  195100   75195 1490521], Length=75
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: range(20149, 40298)
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: C

In [16]:
combined_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,1672.0,20149.0,0.003318,0.000180
1,1672.0,20150.0,0.003736,0.000200
2,1672.0,20151.0,0.003329,0.000180
3,1672.0,20152.0,0.003194,0.000179
4,1672.0,20153.0,0.003431,0.000184
5,1672.0,20154.0,0.003205,0.000173
6,1672.0,20155.0,0.003600,0.000194
7,1672.0,20156.0,0.003476,0.000189
8,1672.0,20157.0,0.003634,0.000198
9,1672.0,20158.0,0.003205,0.000183


Below you can see how transformer works even without train set.

In [17]:
config = CombinedConfig(train_ts=None, val_ts=500, test_ts=None, val_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:45,299][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:45,335][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:45,338][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 500/500 [00:00<00:00, 1826.19it/s]
[2025-08-17 15:46:45,614][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: None
        Val time series IDs: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Test time series IDs: None
    Time periods
        Train time periods: None
        Val time periods: range(0, 20149)
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers part

In [18]:
combined_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,4380.0,0.0,0.001253,0.000005
1,4380.0,1.0,0.001162,0.000004
2,4380.0,2.0,0.000756,0.000003
3,4380.0,3.0,0.000959,0.000004
4,4380.0,4.0,0.001219,0.000005
5,4380.0,5.0,0.001275,0.000005
6,4380.0,6.0,0.001456,0.000006
7,4380.0,7.0,0.001196,0.000004
8,4380.0,8.0,0.001433,0.000006
9,4380.0,9.0,0.001072,0.000004


##### Partial fitting on train set

Makes already fitted transformer to be fitted on new train set too. Must implement `partial_fit`.

In [19]:
config = CombinedConfig(train_ts=500, val_ts=500, test_ts=None, train_time_period=0.5, val_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, partial_fit_initialized_transformer=True, nan_threshold=0.5, random_state=999)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:45,798][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:45,864][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:45,867][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1600.96it/s]
[2025-08-17 15:46:46,183][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 500/500 [00:00<00:00, 1323.93it/s]
[2025-08-17 15:46:46,563][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDs: [ 252575  483796  482558 1490521  471658 ... 450863 242764  10196 405441 792059], Length=75
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: range(20149, 40298)
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: C

In [20]:
combined_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,252575.0,20149.0,0.000102,3.136682e-06
1,252575.0,20150.0,0.000124,1.312285e-06
2,252575.0,20151.0,0.000113,2.464536e-06
3,252575.0,20152.0,0.000147,7.681670e-07
4,252575.0,20153.0,0.000203,3.200696e-06
5,252575.0,20154.0,0.000169,1.984431e-06
6,252575.0,20155.0,0.000147,2.976647e-06
7,252575.0,20156.0,0.000079,3.840835e-07
8,252575.0,20157.0,0.000113,1.344292e-06
9,252575.0,20158.0,0.000113,1.152250e-06
